In [1]:
import numpy as np
import pandas as pd
import gc
import seaborn as sns
import xgboost as xgb
import lightgbm as lgb
import math
import lightgbm as lgb
from collections import Counter  
import time
from scipy.stats import kurtosis,iqr
import seaborn as sns
from scipy import ptp
from tqdm import tqdm
from datetime import timedelta
import datetime
from sklearn.metrics import accuracy_score, roc_auc_score,log_loss,f1_score
from sklearn.model_selection import KFold,StratifiedKFold
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import KFold
import matplotlib.pyplot as plt
from datetime import datetime,timedelta
from sklearn.preprocessing import MinMaxScaler
import warnings
import os
from sklearn.utils import shuffle
from sklearn.externals import joblib
from mpl_toolkits.mplot3d import Axes3D
pd.options.display.max_columns = None
pd.options.display.max_rows = None
import warnings
warnings.filterwarnings('ignore')

In [2]:
### 北京市重点区域信息
cols = ['id', 'area_name', 'area_type', 'Center_x', 'Center_y', 'Grid_x', 'Grid_y', 'area']
area_passenger_info = pd.read_csv('/home/poac/AnomalyDetectionDataset/Block_predict/area_passenger_info.csv', names=cols)

In [3]:
### 读取北京市迁徙指数
migration_index = pd.read_csv('/home/poac/AnomalyDetectionDataset/Block_predict/processed/migration_index.csv')
migration_index.iloc[0:5]

,date,migration_in_index,migration_out_index
0,2020-01-17,372,377
1,2020-01-18,371,375
2,2020-01-19,371,379
3,2020-01-20,371,382
4,2020-01-21,371,382


In [4]:
### 北京市网格人流量指数,200*200网格
cols = ['date', 'hour', 'Grid_x_1', 'Grid_y_1', 'index_1']
short_stay = pd.read_csv('/home/poac/AnomalyDetectionDataset/Block_predict/shortstay_20200201_20200215.csv', names=cols, encoding='gb2312', sep='\t')
tem = pd.read_csv('/home/poac/AnomalyDetectionDataset/Block_predict/shortstay_20200117_20200131.csv', names=cols, encoding='gb2312', sep='\t')
short_stay = pd.concat([short_stay, tem], axis=0)
del tem

In [5]:
short_stay['date'] = short_stay['date'].apply(lambda x: ''.join(str(x)[0:4]+'-'+str(x)[4:6]+'-'+str(x)[6:8]))

In [6]:
short_stay.iloc[0:5]

,date,hour,Grid_x_1,Grid_y_1,index_1
0,2020-02-01,10,116.499307,39.806222,15.7
1,2020-02-01,1,116.384348,39.939035,24.7
2,2020-02-01,0,116.170575,40.049517,16.3
3,2020-02-01,20,116.443470,39.910450,1.8
4,2020-02-01,15,116.588958,39.780357,0.3


In [7]:
### 读取sub
sub = pd.read_csv('/home/poac/code/Block_predict/submit/test_submit_example.csv', names=['id', 'date/hour', 'index'])
sub = sub.merge(area_passenger_info, how='left', on=['id'])
sub['date'] = sub['date/hour'].apply(lambda x: ''.join(str(x)[0:4]+'-'+str(x)[4:6]+'-'+str(x)[6:8]))
sub['date'] = pd.to_datetime(sub['date'])
sub['hour'] = sub['date/hour'].apply(lambda x: int(str(x)[8:]))
sub = sub.drop(['date/hour'], axis=1)

In [35]:
### 读取区域人流量指数，含标签
area_passenger_index_chunk = pd.read_csv('/home/poac/AnomalyDetectionDataset/Block_predict/processed/area_passenger_index.csv', chunksize=10000)

In [13]:
area_passenger_index.iloc[0:5]

,id,index,date,hour,area_name,area_type,Center_x,Center_y,Grid_x,Grid_y,area,label
0,1,1.8,2020-01-17,0,北坞公园,旅游景点;公园,116.256586,39.986913,116.256713,39.987525,395925.609375,2.0
1,1,1.5,2020-01-17,1,北坞公园,旅游景点;公园,116.256586,39.986913,116.256713,39.987525,395925.609375,1.5
2,1,1.3,2020-01-17,2,北坞公园,旅游景点;公园,116.256586,39.986913,116.256713,39.987525,395925.609375,1.8
3,1,1.3,2020-01-17,3,北坞公园,旅游景点;公园,116.256586,39.986913,116.256713,39.987525,395925.609375,1.3
4,1,1.7,2020-01-17,4,北坞公园,旅游景点;公园,116.256586,39.986913,116.256713,39.987525,395925.609375,1.4


In [36]:
### 找对area_passenger_index中重点区域中心最近的坐标，并将其index添加
### 先将area_passenger_index和short_stay merge
area_passenger_index = pd.DataFrame()
for chunk in tqdm(area_passenger_index_chunk):
    chunk = chunk.merge(short_stay, how='left', on=['date', 'hour'])
    chunk['distance'] = (chunk['Grid_x']-chunk['Grid_x_1'])**2 + (chunk['Grid_y']-chunk['Grid_y_1'])**2
    chunk['month'] = chunk['date'].apply(lambda x: int(str(x)[5:7]))
    chunk['day'] = chunk['date'].apply(lambda x: int(str(x)[8:]))
    chunk  = chunk.drop(['date'], axis=1)
    ### 按照month,day,hour,distance排序
    chunk= chunk.sort_values(by=['month', 'day', 'hour', 'distance'], ascending=[True, True, True, True])
    ### 按照month,day,hour多列去重复，取第一次出现的数据，实现最小距离的获取
    chunk = chunk.drop_duplicates(subset=['month','day', 'hour'],keep='first')
    ### 处理一个chunk之后合并
    area_passenger_index = pd.concat([area_passenger_index, chunk], axis=0)



0it [00:00, ?it/s]

MemoryError: 

In [ ]:
area_passenger_index.iloc[0:10]

In [ ]:
### 处理完之后还需要再一次排序去重

In [2]:
### 读取input_for_sub
input_for_sub = pd.read_csv('/home/poac/AnomalyDetectionDataset/Block_predict/processed/input_for_sub.csv')

In [4]:
input_for_sub.shape

(215352, 12)

In [72]:
### 对训练数据集进行处理
def get_data(df):
    ### 从北京市迁徙指数中，按天数，添加当前的迁徙指数
    df = df.merge(migration_index, how='left', on=['date'])
    df = df.reset_index(drop=True)
    
     ### 将北京市网格人流量数据添加进来
    df['short_stay'] = 0
    ### 使用月份，日期，小时，进行初步筛选
    for index, row in tqdm(df.iterrows()):
        tem = short_stay[(short_stay['date']==row['date']) & (short_stay['hour']==row['hour'])]
        distance = np.zeros((tem.shape[0], 1))
        distance = pd.DataFrame(distance)
        distance.columns = ['dis']
        ### 和所有样本计算距离
        distance['dis'] = (tem['Grid_x']-row['Grid_x'])**2 + (tem['Grid_y']-row['Grid_y'])**2
        min_dis = distance['dis'].min()
        print(min_dis)
        ### 取出最小值对应的index
        print('tem shape:', tem.shape)
        print('distance shape:', distance.shape)
        a= tem[distance['dis']==min_dis]['index']
        df.iloc[index]['short_stay'] = tem[distance['dis']==min_dis]['index'].values[0]
        
        
    ### 将date变成月份和天数
    df['month'] = df['date'].apply(lambda x: int(str(x)[5:7]))
    df['day'] = df['date'].apply(lambda x: int(str(x)[8:]))
    ### 删除原来的date，id，area_name，area_type
    df  = df.drop(['date', 'id', 'area_name', 'area_type'], axis=1)
    ### 对坐标和面积做归一化
    sc = MinMaxScaler()
#     sc.fit(df[['Center_x', 'Center_y', 'Grid_x', 'Grid_y', 'area']])
#     df[['Center_x', 'Center_y', 'Grid_x', 'Grid_y', 'area']] = sc.transform(df[['Center_x', 'Center_y', 'Grid_x', 'Grid_y', 'area']])
    
    sc.fit(df[['area']])
    df[['area']] = sc.transform(df[['area']])
    ### 将label=-999的数据删除，这部分数据由于时间太靠近线上，没有标签
    df = df[df['label']!=-999].reset_index(drop=True)
    ### 将特征和label分离
    labels = np.zeros((df.shape[0], 1))
    labels = pd.DataFrame(labels)
    labels.columns = ['label']
    labels = df['label']
    ### 删除原始数据中的label
    df = df.drop(['label'], axis=1)

    
    
    ### 返回特征,标签,归一化句柄
    return df, labels, sc

In [73]:
processed, label, sc = get_data(df=area_passenger_index)

0it [00:00, ?it/s]

2.941620286627449e-05
tem shape: (95652, 5)
distance shape: (95652, 1)


IndexingError: Unalignable boolean Series provided as indexer (index of the boolean Series and of the indexed object do not match

In [12]:
processed.shape

(502488, 11)

In [11]:
processed.iloc[0:5]

,index,hour,Center_x,Center_y,Grid_x,Grid_y,area,migration_in_index,migration_out_index,month,day
0,1.8,0,116.256586,39.986913,116.256713,39.987525,0.017824,372,377,1,17
1,1.5,1,116.256586,39.986913,116.256713,39.987525,0.017824,372,377,1,17
2,1.3,2,116.256586,39.986913,116.256713,39.987525,0.017824,372,377,1,17
3,1.3,3,116.256586,39.986913,116.256713,39.987525,0.017824,372,377,1,17
4,1.7,4,116.256586,39.986913,116.256713,39.987525,0.017824,372,377,1,17


In [12]:
### 对线上提交数据进行处理
def get_sub(df, sc):
    ### 从北京市迁徙指数中，按天数，添加当前的迁徙指数
    df = df.merge(migration_index, how='left', on=['date'])
    ### 将date变成月份和天数
    df['month'] = df['date'].apply(lambda x: int(str(x)[5:7]))
    df['day'] = df['date'].apply(lambda x: int(str(x)[8:]))
    ### 删除原来的date，id，area_name，area_type
    df  = df.drop(['date', 'id', 'area_name', 'area_type'], axis=1)
    ### 对坐标和面积做归一化
    df[['area']] = sc.transform(df[['area']])
    
#     df[['Center_x', 'Center_y', 'Grid_x', 'Grid_y', 'area']] = sc.transform(df[['Center_x', 'Center_y', 'Grid_x', 'Grid_y', 'area']])

    ### 删除原始数据中的label
    df = df.drop(['label'], axis=1)
    return df

In [13]:
input_for_sub = get_sub(df=input_for_sub, sc=sc)

In [14]:
input_for_sub.iloc[0:5]

,index,hour,Center_x,Center_y,Grid_x,Grid_y,area,migration_in_index,migration_out_index,month,day
0,0.6,0,116.256586,39.986913,116.256713,39.987525,0.017824,353,341,2,7
1,0.7,1,116.256586,39.986913,116.256713,39.987525,0.017824,353,341,2,7
2,1.6,2,116.256586,39.986913,116.256713,39.987525,0.017824,353,341,2,7
3,0.5,3,116.256586,39.986913,116.256713,39.987525,0.017824,353,341,2,7
4,0.9,4,116.256586,39.986913,116.256713,39.987525,0.017824,353,341,2,7


In [15]:
### K折交叉验证训练模型
### 定义模型参数
params = {'num_leaves': 38,
          'min_data_in_leaf': 50,
          'objective': 'regression',
          'max_depth': -1,
          'learning_rate': 0.02,
          "min_sum_hessian_in_leaf": 6,
          "boosting": "gbdt",
          "feature_fraction": 0.9,
          "bagging_freq": 1,
          "bagging_fraction": 0.7,
          "bagging_seed": 11,
          "lambda_l1": 0.1,
          "verbosity": -1,
          "nthread": 4,
          'metric': 'mae',
          "random_state": 2019,
#           'device': 'gpu'
          }

In [16]:
def r_mean_absolute_percentage_error(y_true, y_pred):
    return np.sqrt(np.mean((y_true-y_pred)**2))

In [17]:
folds = KFold(n_splits=5, shuffle=True, random_state=2019)

In [18]:
### K 折
for fold_, (trn_idx, val_idx) in enumerate(folds.split(processed)):
    print("fold {}".format(fold_ + 1))
    trn_data = lgb.Dataset(processed.iloc[trn_idx], label=label.iloc[trn_idx])
    val_data = lgb.Dataset(processed.iloc[val_idx], label=label.iloc[val_idx])
    num_round = 1000
    clf = lgb.train(params,
                    trn_data,
                    num_round,
                    valid_sets=[trn_data, val_data],
                    verbose_eval=200,
                    early_stopping_rounds=200)
    pred = clf.predict(processed.iloc[val_idx], num_iteration=clf.best_iteration)
    print('RMSE:', r_mean_absolute_percentage_error(y_true=label.iloc[val_idx], y_pred=pred))
    print('Score:', 1/(1+r_mean_absolute_percentage_error(y_true=label.iloc[val_idx], y_pred=pred)))

fold 1
Training until validation scores don't improve for 200 rounds
[200]	training's l1: 3.95862	valid_1's l1: 3.95712
[400]	training's l1: 3.42626	valid_1's l1: 3.43949
[600]	training's l1: 3.21269	valid_1's l1: 3.23577
[800]	training's l1: 3.05786	valid_1's l1: 3.09229
[1000]	training's l1: 2.93256	valid_1's l1: 2.9763
Did not meet early stopping. Best iteration is:
[1000]	training's l1: 2.93256	valid_1's l1: 2.9763
RMSE: 6.388601326142006
Score: 0.13534361320346874
fold 2
Training until validation scores don't improve for 200 rounds
[200]	training's l1: 3.93684	valid_1's l1: 3.98217
[400]	training's l1: 3.40788	valid_1's l1: 3.45861
[600]	training's l1: 3.20384	valid_1's l1: 3.2622
[800]	training's l1: 3.04917	valid_1's l1: 3.11366
[1000]	training's l1: 2.91265	valid_1's l1: 2.98264
Did not meet early stopping. Best iteration is:
[1000]	training's l1: 2.91265	valid_1's l1: 2.98264
RMSE: 6.582981745336346
Score: 0.13187424598707703
fold 3
Training until validation scores don't impro

In [19]:
### submit
submit = pd.read_csv('/home/poac/code/Block_predict/submit/test_submit_example.csv', names=['id', 'date/hour', 'index'])
submit = submit.merge(area_passenger_info, how='left', on=['id'])

In [20]:
submit.iloc[0:5]

,id,date/hour,index,area_name,area_type,Center_x,Center_y,Grid_x,Grid_y,area
0,1,2020021600,1.5,北坞公园,旅游景点;公园,116.256586,39.986913,116.256713,39.987525,395925.609375
1,1,2020021601,1.5,北坞公园,旅游景点;公园,116.256586,39.986913,116.256713,39.987525,395925.609375
2,1,2020021602,1.5,北坞公园,旅游景点;公园,116.256586,39.986913,116.256713,39.987525,395925.609375
3,1,2020021603,1.5,北坞公园,旅游景点;公园,116.256586,39.986913,116.256713,39.987525,395925.609375
4,1,2020021604,1.5,北坞公园,旅游景点;公园,116.256586,39.986913,116.256713,39.987525,395925.609375


In [21]:
def get_submit(df, pred):
    ### 只保留id，日期，小时
    df = df[['id', 'date/hour']]
    ### 转换日期格式
    ### 将预测结果合并
    df = pd.concat([df, pred], axis=1)
    return df

In [22]:
### 预测
pred = clf.predict(input_for_sub)
pred = pd.DataFrame(pred)
pred.columns = ['pred']

In [23]:
submit = get_submit(df=submit, pred=pred)

In [24]:
submit.iloc[0:5]

,id,date/hour,pred
0,1,2020021600,0.661181
1,1,2020021601,0.759125
2,1,2020021602,1.699909
3,1,2020021603,0.414776
4,1,2020021604,0.760199


In [25]:
### 保存
submit.to_csv('/home/poac/code/Block_predict/submit/block_predict_result.csv', index=None, header=None)